In [1]:
#libraries for multi-variate encoder-decoder lstm
from pandas_datareader import data
import matplotlib.pyplot as plt
from math import sqrt

import pandas as pd
from pandas import read_csv
import datetime as dt
import urllib.request, json
import os
import numpy as np
from numpy import newaxis, split, array
import matplotlib.pyplot as plt
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import time
import warnings
from keras.layers.core import Dense, Activation, Dropout

from keras.layers import Convolution1D, MaxPooling1D, Flatten,  Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from datetime import datetime
from keras.layers import Dense, Flatten, LSTM, RepeatVector, TimeDistributed


Using TensorFlow backend.


In [2]:
#Load data

In [3]:
#Asset data
def GetData(fileName):
    return pd.read_csv(fileName, header=0,usecols=['Date','Adj Close'], parse_dates=True, index_col='Date')
    
    

In [4]:
data_ivv = GetData('Asset_Dataset/IVV_daily.csv')
data_ivv.columns = ['IVV Adjusted Close']
%store data_ivv


Stored 'data_ivv' (DataFrame)


In [5]:
#normalize ivv
scaler = StandardScaler()
data_ivv_norm = scaler.fit_transform(data_ivv.values.reshape(-1, 1))


In [6]:
data_ivv_norm

array([[-1.32399803],
       [-1.32692451],
       [-1.31838748],
       ...,
       [ 1.50392089],
       [ 1.52092371],
       [ 1.42457504]])

In [7]:
#change to df for merging
data_ivv_norm_df = pd.DataFrame(data_ivv_norm)

In [8]:
#market and econ data
%store -r scaled_macro_market
scaled_macro_market

array([[0.        , 0.03274277, 0.02521008, 1.        , 0.52      ,
        0.13794926],
       [0.        , 0.03274277, 0.02521008, 1.        , 0.52      ,
        0.13794926],
       [0.        , 0.03274277, 0.02521008, 1.        , 0.52      ,
        0.13794926],
       ...,
       [1.        , 0.99159004, 0.97478992, 0.        , 0.68      ,
        0.48625788],
       [1.        , 0.99159004, 0.9789916 , 0.        , 0.68      ,
        0.48625788],
       [1.        , 1.        , 0.9789916 , 0.0625    , 0.72      ,
        0.48625788]])

In [9]:
scaled_macro_market_df = pd.DataFrame(scaled_macro_market)

In [10]:
merged_data = pd.concat([data_ivv_norm_df,scaled_macro_market_df],axis=1)
merged_data = merged_data.dropna()


In [11]:
merged_data.columns = ['IVV adj close','Real GDP','CPI All','Fed Rate','Unemployment rate','Personal Savings Rate','VIX']

In [12]:
merged_data

,IVV adj close,Real GDP,CPI All,Fed Rate,Unemployment rate,Personal Savings Rate,VIX
0,-1.323998,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
1,-1.326925,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
2,-1.318387,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
3,-1.286918,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
4,-1.262767,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
5,-1.257402,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
6,-1.258376,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
7,-1.273745,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
8,-1.308874,0.000000,0.032743,0.025210,1.0000,0.52,0.137949
9,-1.293505,0.000000,0.032743,0.025210,1.0000,0.52,0.137949


In [13]:
##LSTM STARTS HERE##

In [14]:
len(merged_data)

1259

In [15]:
input_data = merged_data.values

In [16]:
#try using plotly / seaborn / iplot
#to plot IVV & All parameters


#


In [23]:
#The multiple input features make this an multivariate regression problem
lookback = 50

test_size = int(0.3*len(merged_data))
X=[]
y=[]
for i in range(len(merged_data)-lookback-1):
    t=[]
    for j in range(0,lookback):
    
        t.append(input_data[[(i+j)],:])
    X.append(t)
    y.append(input_data[i+lookback,1])

In [34]:
#transform input data into LSTM expected structure
#first convery X into array then reshape()

X,y = np.array(X), np.array(y)
X_test = X[:test_size+lookback]

X = X.reshape(X.shape[0]*350,lookback,2)
X_test = X_test.reshape(X_test.shape[0]*500, lookback , 2)
print(X.shape)
print(X_test.shape)


ValueError: cannot reshape array of size 422800 into shape (422800,50,2)

In [ ]:
model = Sequential()
model.add(LSTM(units=30, return_sequences= True, input_shape=(X.shape[1],2)))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X, y, epochs=200, batch_size=32)

In [ ]:
predicted_value= model.predict(X_test)

In [ ]:
plt.plot(predicted_value, color= 'red')
plt.plot(input_data[lookback:test_size+(2*lookback),1], color='green')
plt.title("Opening price of stocks sold")
plt.xlabel("Time (latest-> oldest)")
plt.ylabel("Stock Opening Price")
plt.show()